In [2]:
prompt = """BẠN LÀ TRỢ LÝ AI TƯ VẤN GIAO THÔNG THÔNG MINH.

NHIỆM VỤ CHÍNH:
- Phân tích câu hỏi của người dùng để hiểu ý định cụ thể
- Nếu hỏi thông tin về tuyến đường nào đó thì buộc phải đưa đầy đủ thông tin về số lượng và tốc độ của oto lẫn xe máy
- Đưa ra lời khuyên và phân tích dựa trên dữ liệu giao thông 
- Khi cần thiết, sử dụng tools để lấy thông tin và hình ảnh thực tế

NGUYÊN TẮC PHÂN LOẠI:
   - Nhiều xe + vận tốc < 12 km/h → Ùn tắc
   - Nhiều xe + vận tốc 12-30 km/h → Đông
   - Vận tốc >= 30 km/h → Thông thoáng
   - Ít xe + vận tốc thấp → Chậm nhưng không tắc

KHI TRẢ LỜI:
- Luôn cung cấp thông tin chi tiết và chính xác
- Nếu có hình ảnh từ camera, hãy mô tả tình hình giao thông dựa trên hình ảnh
- Đưa ra lời khuyên cụ thể dựa trên dữ liệu thực tế
"""

In [44]:
import dotenv
dotenv.load_dotenv()
from tool_func import get_frame_road, get_info_road
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
agent = create_react_agent(model= llm, tools= [get_frame_road, get_info_road], 
                           prompt= prompt)
config = {"configurable": {"thread_id": "1"}}


In [ ]:
str = "cho tôi xin thông tin về Đường Láng, cả ảnh nữa nhé"
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": str}]},
    config
)

In [46]:
str = "cho tôi xin thông tin về Văn Phú, cả ảnh nữa nhé"
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": str}]},
    # config= {"configurable": {"thread_id": "2"}}
)

In [82]:
sf_response['messages'][-1].type

'ai'

In [107]:
import dotenv
dotenv.load_dotenv()

from tool_func import get_frame_road, get_info_road
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

prompt = """BẠN LÀ TRỢ LÝ AI TƯ VẤN GIAO THÔNG THÔNG MINH.

NHIỆM VỤ CHÍNH:
- Phân tích câu hỏi của người dùng để hiểu ý định cụ thể
- Nếu hỏi thông tin về tuyến đường nào đó thì buộc phải đưa đầy đủ thông tin về số lượng và tốc độ của oto lẫn xe máy
- Đưa ra lời khuyên và phân tích dựa trên dữ liệu giao thông 
- Khi cần thiết, sử dụng tools để lấy thông tin và hình ảnh thực tế

NGUYÊN TẮC PHÂN LOẠI:
   - Nhiều xe + vận tốc < 12 km/h → Ùn tắc
   - Nhiều xe + vận tốc 12-30 km/h → Đông
   - Vận tốc >= 30 km/h → Thông thoáng
   - Ít xe + vận tốc thấp → Chậm nhưng không tắc

KHI TRẢ LỜI:
- Luôn cung cấp thông tin chi tiết và chính xác
- Nếu có hình ảnh từ camera, hãy mô tả tình hình giao thông dựa trên hình ảnh
- Đưa ra lời khuyên cụ thể dựa trên dữ liệu thực tế
"""

class ChatBotAgent:
    def __init__(self):
        self.prompt = prompt
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
        self.agent = create_react_agent(model= self.llm, tools= [get_frame_road, get_info_road], 
                               prompt= prompt)
    def process_response(self, sf_response: dict):
        data = {
            'image': [],
            'text': None
        }
        for msg in sf_response['messages']:
            if msg.type == 'tool':
                if msg.name == 'get_frame_road':
                    data['image'].append(msg.content)
        data['text'] = sf_response['messages'][-1].content
        return data
    
    def get_response(self, user_input):
        sf_response = self.agent.invoke(
            {"messages": [{"role": "user", "content": user_input}]}
        )
        return self.process_response(sf_response)

In [108]:
chat = ChatBotAgent()

In [115]:
mess = "cho tôi ảnh tuyến Đường Láng"
res = chat.get_response(mess)

In [116]:
res

{'image': ['...'],
 'text': 'Tuyến Đường Láng hiện đang:\n- Có 50 xe máy với vận tốc trung bình 15 km/h\n- Có 20 ô tô với vận tốc trung bình 10 km/h\n\n**Tình hình giao thông:**\nDựa trên hình ảnh và dữ liệu, Đường Láng hiện đang **Đông**. Mặc dù có nhiều xe máy và ô tô, nhưng vận tốc trung bình của xe máy là 15 km/h và ô tô là 10 km/h, nằm trong khoảng 12-30 km/h, cho thấy tình trạng đông đúc nhưng vẫn di chuyển được.\n\n**Lời khuyên:**\nNếu bạn có thể, hãy cân nhắc lựa chọn lộ trình thay thế hoặc hoãn chuyến đi qua Đường Láng vào thời điểm hiện tại để tránh tình trạng đông đúc.'}